##  자동 기상 스크래핑

#### ※ 웹 크롤링 및 스크래핑, csv 저장이 동시에 이뤄지기 때문에 작업 중지를 방지하기 위해 단독 실행을 추천합니다.

### Chromedriver

In [ ]:
"""
크롬 드라이버는 사용자의 크롬 버전에 맞게 드라이버를 다운로드 받아 스크래핑 코드와 같은 폴더에 넣어주시면 됩니다. 

크롬 브라우저의 버전은 우측 상단 더보기란 -> 도움말 -> Chrome 정보 를 선택하면 확인할 수 있습니다.

https://chromedriver.chromium.org/downloads 에 가시면 버전에 맞는 크롬 드라이버를 찾아서 다운로드할 수 있습니다.
"""

### Install Selenium

In [1]:
!pip3 install selenium

### 화면 가로 확장

In [7]:
# 화면 가로 확장 코드 (기본 width 50%)
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:70% !important; }</style>"))

### 데이터 저장을 위한 폴더 생성

In [6]:
import os
try:
    os.mkdir('./Weather_Data_CSV')
except FileExistsError:
    print('해당 폴더가 이미 존재합니다.\n폴더를 확인해주세요.')

### 정상 시각 일치를 위한 시각과 48개의 순번 Dict

In [8]:
# 11:20 PM부터 데이터가 존재하는 날을 위한 시간 순차 번호
clk_11_48 = {'11:50 PM':1, '12:20 AM':2, '12:50 AM':3, '1:20 AM':4, '1:50 AM':5, '2:20 AM':6,
         '2:50 AM':7, '3:20 AM':8, '3:50 AM':9, '4:20 AM':10, '4:50 AM':11, '5:20 AM':12,
         '5:50 AM':13, '6:20 AM':14, '6:50 AM':15, '7:20 AM':16, '7:50 AM':17, '8:20 AM':18,
         '8:50 AM':19, '9:20 AM':20, '9:50 AM':21, '10:20 AM':22, '10:50 AM':23, '11:20 AM':24,
         '11:50 AM':25, '12:20 PM':26, '12:50 PM':27, '1:20 PM':28, '1:50 PM':29, '2:20 PM':30,
         '2:50 PM':31, '3:20 PM':32, '3:50 PM':33, '4:20 PM':34, '4:50 PM':35, '5:20 PM':36,
         '5:50 PM':37, '6:20 PM':38, '6:50 PM':39, '7:20 PM':40, '7:50 PM':41, '8:20 PM':42,
         '8:50 PM':43, '9:20 PM':44, '9:50 PM':45, '10:20 PM':46, '10:50 PM':47, '11:20 PM':48}

# reversed
re_clk_11_48 = dict(map(reversed, clk_11_48.items()))
print(re_clk_11_48)
print()

{1: '11:50 PM', 2: '12:20 AM', 3: '12:50 AM', 4: '1:20 AM', 5: '1:50 AM', 6: '2:20 AM', 7: '2:50 AM', 8: '3:20 AM', 9: '3:50 AM', 10: '4:20 AM', 11: '4:50 AM', 12: '5:20 AM', 13: '5:50 AM', 14: '6:20 AM', 15: '6:50 AM', 16: '7:20 AM', 17: '7:50 AM', 18: '8:20 AM', 19: '8:50 AM', 20: '9:20 AM', 21: '9:50 AM', 22: '10:20 AM', 23: '10:50 AM', 24: '11:20 AM', 25: '11:50 AM', 26: '12:20 PM', 27: '12:50 PM', 28: '1:20 PM', 29: '1:50 PM', 30: '2:20 PM', 31: '2:50 PM', 32: '3:20 PM', 33: '3:50 PM', 34: '4:20 PM', 35: '4:50 PM', 36: '5:20 PM', 37: '5:50 PM', 38: '6:20 PM', 39: '6:50 PM', 40: '7:20 PM', 41: '7:50 PM', 42: '8:20 PM', 43: '8:50 PM', 44: '9:20 PM', 45: '9:50 PM', 46: '10:20 PM', 47: '10:50 PM', 48: '11:20 PM'}



### 변수 및 단계별 알고리즘 설명

In [9]:
"""
변수 h => 하루를 30분 단위로 쪼갠 온도 개수, 데이터의 개수를 h로 설정해서 반복시켰는데, 
            48까지 못 채우는 경우도 있음.

변수 x => 하루 데이터의 시작 시각인 11:50 PM보다 먼저 이른 시각부터 시작했을 경우 중에 11:50 PM 의 데이터가
            결손되었을 경우 반복을 통해 비결손 데이터전까지 처리를 하기 위한 변수.
        
변수 h3 => 2.1.1.1.의 경우에서 하루 동안 48개의 데이터를 스크래핑하기 위한 카운트 변수.

변수 z => 2.1.1.2.의 경우에서 비결손 정상 데이터 지점까지 결손된 시각의 반복 처리 횟수를 결정하기 위한 변수.

======================================================================================================================================================

1. 반복을 위한 month 딕셔너리와 글로벌 변수 지정
    -> 각 달마다 일수가 다르고 하루 단위로 동작되는 코드는 중복되는 내용이 많기 때문에 간소화를 위해 월 Dict 이용.
        
2. 해당 월의 일수 동안 반복하도록 하는 월단위 while문 진입
    -> 연, 월, 일 변수의 조합으로 해당 날짜(최소 단위 1일)의 기상 주소 접속.
    -> 변수 h, h2 초기화.
    
    2.1. 11:50 PM 이전 시각부터 시작하는 경우
            -> 42는 적당히 오차 범위일거라 생각하는 숫자로 임의 지정한 것이지만 42 ~ 48 까지는 정상 시각 시작전을 뜻함.
            -> 변수 x 초기화.
        
        2.1.1. 하루의 기상 데이터 시작 시각의 경우에 따른 while문 진입
                -> 변수 x의 값에 따라 경우가 나뉘는데 크게 x==1인 경우(2.1.1.1.)와 x>=2인 경우(2.1.1.2.)로 나눠서 설명.
                -> 중복이 많아서 clk_11_48[clk_str]로 처리하려 했으나 이전 시각대에서 47같은 숫자가 나와서 하나씩 따로 설정함.   
                
            2.1.1.1. 11:50 PM의 이전 시각부터 해당일의 데이터가 시작하고, 11:50 PM의 데이터도 존재할 경우
                       -> h3 변수 지정.
                       -> 11:50 PM을 기준으로 하루 48개의 데이터를 스크래핑하는 while문 진입.
                       
            ---------------------------------------------------------------------------------------------------------------------------------
            |    2.1.1.1.1. 사이트에서 시각과 온도를 스크랩하도록 Try문 실행
            |                -> 5:01 과 같은 규칙 외 시각을 걸러내기 위해 dict 안에 있는지 검사.
            |                   이상 시각 검출시에는 변수 h2를 1만 증가시켜서 다음 시각으로 이동.
            |            
            |        2.1.1.1.1.1. 순차적으로 증가중인 h3와 현재 시각 문자열을 clk순번 dict 출력 숫자로 비교해봤을때 일치하는 경우 (=시각이 정상적인 경우)
 <Core>     |                    -> 시각, 순번, 온도 정보로 csv 파일 저장 후 변수 h2, h3 증가.
            |                    
            |        2.1.1.1.1.2. 다음 순서로 올 h3와 현재 시각이 일치하지 않는 경우 (=데이터 결손)
            |                    -> 결손 데이터 칸의 정상시각을 입력후 전 시각의 온도를 그대로 저장.
            |                    
            |    2.1.1.1.2. 정상 데이터수를 채우지 못하고 NoSuchElementException 로 인해 조기 종료될 경우
            |                -> 예외 처리를 통하여 해당 경우로 진입하고 변수 h가 48이 될때까지(=하루 데이터 갯수 충족) 결손된 데이터를 정상시각까지 처리.
            ---------------------------------------------------------------------------------------------------------------------------------              
            
            2.1.1.2. 11:50 PM의 이전 시각부터 해당일의 데이터가 시작하고, 11:50 PM의 데이터가 존재하지 않을 경우
                        -> 결손데이터가 지속되다가 데이터가 존재하는 지점까지 가게 되는데 이때 변수 z를 이용해 정상지점까지 처리 반복.
                        -> h3 = 1 + (x-1)의 계산으로 결손 데이터를 반복한뒤 직후 지점부터 하루 끝 지점까지 반복하도록 설정.
                
                2.1.1.2.1. 이후 동작은 <Core>와 동일한 구조로 실행
             
    2.2. 11:50 PM 부터 하루 데이터가 시작하는 경우
        <Core> 동작 실행. 시작 시각이 정상적이기 때문에 중간 결손데이터 처리만 하면 되므로 단계가 간단함.
    
"""

'\n변수 h => 하루를 30분 단위로 쪼갠 온도 개수, 데이터의 개수를 h로 설정해서 반복시켰는데, \n            48까지 못 채우는 경우도 있음.\n        \n변수 x => 하루 데이터의 시작 시각인 11:50 PM보다 먼저 이른 시각부터 시작했을 경우 중에 11:50 PM 의 데이터가\n            결손되었을 경우 반복을 통해 비결손 데이터전까지 처리를 하기 위한 변수.\n        \n변수 h3 => 2.1.1.1.의 경우에서 하루 동안 48개의 데이터를 스크래핑하기 위한 카운트 변수.\n\n변수 z => 2.1.1.2.의 경우에서 비결손 정상 데이터 지점까지 결손된 시각의 반복 처리 횟수를 결정하기 위한 변수.\n\n======================================================================================================================================================\n\n1. 반복을 위한 month 딕셔너리와 글로벌 변수 지정\n    -> 각 달마다 일수가 다르고 하루 단위로 동작되는 코드는 중복되는 내용이 많기 때문에 간소화를 위해 월 Dict 이용.\n        \n2. 해당 월의 일수 동안 반복하도록 하는 월단위 while문 진입\n    -> 연, 월, 일 변수의 조합으로 해당 날짜(최소 단위 1일)의 기상 주소 접속.\n    -> 변수 h, h2 초기화.\n    \n    2.1. 11:50 PM 이전 시각부터 시작하는 경우\n            -> 42는 적당히 오차 범위일거라 생각하는 숫자로 임의 지정한 것이지만 42 ~ 48 까지는 정상 시각 시작전을 뜻함.\n            -> 변수 x 초기화.\n        \n        2.1.1. 하루의 기상 데이터 시작 시각의 경우에 따른 while문 진입\n                -> 변수 x의 값에 따라 경우가 나

### 월 마다 다른 일수 반영, 48분할 하루의 온도를 스크래핑, 결측값 정상 입력 함수

In [13]:
def Monthly_Temp(YYYY, MM):
    driver.implicitly_wait(10)
    YYYY = y
    MM = m

    # 1.
    month = {1:31, 2:28, 3:31, 4:30, 5:31, 6:30, 7:31, 8:31, 9:30, 10:31, 11:30, 12:31}
    d=0
    
    global temp_t_x
    global temp_c_x
    global temp_t
    global temp_c  
    
    # 2.
    while(d<month[m]):
        d=d+1
        print('====================< %d일 >=========================' %d)
        print()
        driver.get('https://www.wunderground.com/history/daily/gb/london/EGLC/date/'+str(y)+'-'+str(m)+'-'+str(d))
        driver.implicitly_wait(10)
        time.sleep(3)
                
        h = 1 
        h2 = 1
        
        clk = driver.find_element(By.XPATH,'//*[@id="inner-content"]/div[2]/div[1]/div[5]/div[1]/div/lib-city-history-observation/div/div[2]/table/tbody/tr['+str(h2)+']/td[1]/span')
        clk_str = clk.text
        clk_str.split()

        # 2.1. 
        if(int(clk_11_48[clk_str])>=42):
            print('<11:50 PM 이전 시각부터 데이터 시작>')
            x = 0
            
            # 2.1.1.
            while(h2 < 49):
                clk = driver.find_element(By.XPATH,'//*[@id="inner-content"]/div[2]/div[1]/div[5]/div[1]/div/lib-city-history-observation/div/div[2]/table/tbody/tr['+str(h2)+']/td[1]/span')
                clk_str = clk.text
                clk_str.split()

                if(clk_str == '11:50 PM'):
                    x = 1
                    print('x == 1\n')
                
                if(clk_str == '12:20 AM'):
                    x = 2
                    print('x == 2\n')
                
                if(clk_str == '12:50 AM'):
                    x = 3
                    print('x == 3\n')
                    
                if(clk_str == '1:20 AM'):
                    x = 4
                    print('x == 4\n')
                    
                if(clk_str == '1:50 AM'):
                    x = 5
                    print('x == 5\n')
                    
                if(clk_str == '2:20 AM'):
                    x = 6
                    print('x == 6\n')
                    
                if(clk_str == '2:50 AM'):
                    x = 7
                    print('x == 7\n')
                    
                if(clk_str == '3:20 AM'):
                    x = 8
                    print('x == 8\n')
                    
                if(clk_str == '3:50 AM'):
                    x = 9
                    print('x == 9\n')
                    
                if(clk_str == '4:20 AM'):
                    x = 10
                    print('x == 10\n')

                # 2.1.1.1.
                if(x == 1):
                    h3 = 1
                    
                    while (h3 < 49):
                        
                        # 2.1.1.1.1.
                        try:
                            clk2 = driver.find_element(By.XPATH,'//*[@id="inner-content"]/div[2]/div[1]/div[5]/div[1]/div/lib-city-history-observation/div/div[2]/table/tbody/tr['+str(h2)+']/td[1]/span')
                            clk_str2 = clk2.text
                            clk_str2.split()
                            
                            if(clk_str2 in clk_11_48):
                                
                                # 2.1.1.1.1.1.
                                # 이 if문 진입전에는 h3 = 1로 고정된 상태.
                                if(h3==clk_11_48[clk_str2]):
                                    date_list = [int(y), int(m), int(d)] # 연,월,일 정보로 구성된 날짜 리스트 생성
                                    date_list.append(clk_str2) # 날짜 리스트에 해당 시각 원소 추가
                                    date_list.append(int(h3)) # 해당 시각이 그날 48개로 분할된 시간 중 몇번째인지 원소 추가
                                    temp = driver.find_element(By.XPATH,'//*[@id="inner-content"]/div[2]/div[1]/div[5]/div[1]/div/lib-city-history-observation/div/div[2]/table/tbody/tr['+str(h2)+']/td[2]')
                                    temp_str = temp.text # 스크래핑한 xpath의 온도를 str로 변경
                                    temp_str.split
                                    temp_t=temp_str[:2] # temp_str 에서 온도 숫자만 추출
                                    temp_c=temp_str[2:]
                                    temp_t_x = temp_t
                                    temp_c_x = temp_c
                                    temp_list = date_list 
                                    temp_list.append(temp_t_x) # 날짜+시각 리스트에 온도 원소 추가
                                    temp_list.append(temp_c_x) # 날짜+시각 리스트에 섭씨 기호 원소 추가
                                    time.sleep(0.02)
                                    print(temp_list) # 확인용 print
                                    # 시간 루프 종료 지점

                                    # csv 추가 및 저장
                                    with open("./Weather_Data_CSV/"+str(y)+'_'+str(m)+'_'+"London_Temperature.csv", 'a', newline='') as file:
                                        writer = csv.writer(file)
                                        writer.writerow(temp_list)
                                    print()
                                    h2 = h2+1
                                    h3 = h3+1
                                    
                                # 2.1.1.1.1.2.
                                else:
                                    date_list = [int(y), int(m), int(d)]
                                    print('결손 데이터 칸의 정상 시각 >>',re_clk_11_48[h])
                                    print('사이트에서 스크래핑한 시각 >>', clk_str2)
                                    date_list.append(re_clk_11_48[h])
                                    date_list.append(int(h3))
                                    na_list = date_list
                                    na_list.append(temp_t) # 직전 시각의 온도를 그대로 입력
                                    na_list.append(temp_c)
                                    print(na_list)
                                    h3 = h3+1

                                    with open("./Weather_Data_CSV/"+str(y)+'_'+str(m)+'_'+"London_Temperature.csv", 'a', newline='') as file:
                                        writer = csv.writer(file)
                                        writer.writerow(na_list)
                                    print() 

                                h = h+1 
                                
                            else:
                                for E in range(1):
                                    h2 = h2+1
                                    
                        # 2.1.1.1.2.
                        except NoSuchElementException:
                            if(h<49):
                                # 맨 마지막에 데이터가 없어 조기 종료 되버리는 경우
                                date_list = [int(y), int(m), int(d)]
                                print('결손 데이터 칸의 정상 시각 >>',re_clk_11_48[h]) 
                                print('사이트에서 스크래핑한 시각 >>', clk_str)
                                date_list.append(re_clk_11_48[h])
                                date_list.append(int(h))
                                na_list = date_list
                                na_list.append(temp_t)
                                na_list.append(temp_c)
                                print(na_list)

                                with open("./Weather_Data_CSV/"+str(y)+'_'+str(m)+'_'+"London_Temperature.csv", 'a', newline='') as file:
                                    writer = csv.writer(file)
                                    writer.writerow(na_list)
                                    print() 
                                h = h+1
                            else:
                                break
                    break
                
                # 2.1.1.2.
                if(x >= 2):
                    z = 1
                    while(z < x):
                        date_list = [int(y), int(m), int(d)]
                        print('결손 데이터 칸의 정상 시각 >>',re_clk_11_48[h])
                        print('11:50 PM 이전 시각에 데이터 시작했지만 11:50 PM 데이터 결손')
                        date_list.append(re_clk_11_48[h])
                        date_list.append(z)
                        na_list = date_list
                        na_list.append(temp_t_x) # 직전 시각의 온도를 그대로 입력
                        na_list.append(temp_c_x)
                        print(na_list)
                        
                        print()
                        with open("./Weather_Data_CSV/"+str(y)+'_'+str(m)+'_'+"London_Temperature.csv", 'a', newline='') as file:
                            writer = csv.writer(file)
                            writer.writerow(na_list)
                        z = z+1
                        h = h+1
                        
                    h3 = 1 + (x-1)
                    
                    # 2.1.1.2.1.
                    while (h3 < 49):
                        try:
                            clk2 = driver.find_element(By.XPATH,'//*[@id="inner-content"]/div[2]/div[1]/div[5]/div[1]/div/lib-city-history-observation/div/div[2]/table/tbody/tr['+str(h2)+']/td[1]/span')
                            clk_str2 = clk2.text
                            clk_str2.split()
                            
                            if(clk_str2 in clk_11_48):
                                # 이 if문 진입전에는 h3 = 1로 고정된 상태.
                                if(h3==clk_11_48[clk_str2]):
                                    date_list = [int(y), int(m), int(d)] # 연,월,일 정보로 구성된 날짜 리스트 생성
                                    date_list.append(clk_str2) # 날짜 리스트에 해당 시각 원소 추가
                                    date_list.append(int(h3)) # 해당 시각이 그날 48개로 분할된 시간 중 몇번째인지 원소 추가
                                    temp = driver.find_element(By.XPATH,'//*[@id="inner-content"]/div[2]/div[1]/div[5]/div[1]/div/lib-city-history-observation/div/div[2]/table/tbody/tr['+str(h2)+']/td[2]')
                                    temp_str = temp.text # 스크래핑한 xpath의 온도를 str로 변경
                                    temp_t=temp_str[:2] # temp_str 에서 온도 숫자만 추출
                                    temp_c=temp_str[2:]
                                    temp_t_x = temp_t
                                    temp_c_x = temp_c
                                    temp_list = date_list 
                                    temp_list.append(temp_t_x) # 날짜+시각 리스트에 온도 원소 추가
                                    temp_list.append(temp_c_x) # 날짜+시각 리스트에 섭씨 기호 원소 추가
                                    time.sleep(0.02)
                                    print(temp_list) # 확인용 print
                                    # 시간 루프 종료 지점

                                    # csv 추가 및 저장
                                    with open("./Weather_Data_CSV/"+str(y)+'_'+str(m)+'_'+"London_Temperature.csv", 'a', newline='') as file:
                                        writer = csv.writer(file)
                                        writer.writerow(temp_list)
                                    print()
                                    h2 = h2+1
                                    h3 = h3+1

                                else:
                                    try:
                                        date_list = [int(y), int(m), int(d)]
                                        print('결손 데이터 칸의 정상 시각 >>',re_clk_11_48[h])
                                        # 11:50 이전에 시작했는데 11:50 PM 시각 데이터도 없는 경우는 처리 방법 미해결.
                                        print('사이트에서 스크래핑한 시각 >>', clk_str2)
                                        date_list.append(re_clk_11_48[h])
                                        date_list.append(int(h3))
                                        na_list = date_list
                                        na_list.append(temp_t) # 직전 시각의 온도를 그대로 입력
                                        na_list.append(temp_c)
                                        print(na_list)
                                        h3 = h3+1

                                        with open("./Weather_Data_CSV/"+str(y)+'_'+str(m)+'_'+"London_Temperature.csv", 'a', newline='') as file:
                                            writer = csv.writer(file)
                                            writer.writerow(na_list)
                                        print() 
                                    except KeyError:
                                        break

                                h = h+1
                            
                            else:
                                for E in range(1):
                                    h2 = h2+1

                        except NoSuchElementException:
                            if(h<49):
                                # 맨 마지막에 데이터가 없어 조기 종료 되버리는 경우
                                date_list = [int(y), int(m), int(d)]
                                print('결손 데이터 칸의 정상 시각 >>',re_clk_11_48[h]) 
                                print('사이트에서 스크래핑한 시각 >>', clk_str)
                                date_list.append(re_clk_11_48[h])
                                date_list.append(int(h))
                                na_list = date_list
                                na_list.append(temp_t)
                                na_list.append(temp_c)
                                print(na_list)

                                with open("./Weather_Data_CSV/"+str(y)+'_'+str(m)+'_'+"London_Temperature.csv", 'a', newline='') as file:
                                    writer = csv.writer(file)
                                    writer.writerow(na_list)
                                    print() 
                                h = h+1
                            else:
                                break
                    break
                    
                else:
                    h2=h2+1
                    
                    
        # 2.2.
        else:
            while (h2 < 49):
                try:    
                    clk = driver.find_element(By.XPATH,'//*[@id="inner-content"]/div[2]/div[1]/div[5]/div[1]/div/lib-city-history-observation/div/div[2]/table/tbody/tr['+str(h2)+']/td[1]/span')
                    clk_str = clk.text
                    clk_str.split()

                    if(clk_str in clk_11_48):
                        if(h==clk_11_48[clk_str]):
                            date_list = [int(y), int(m), int(d)] # 연,월,일 정보로 구성된 날짜 리스트 생성
                            date_list.append(clk_str) # 날짜 리스트에 해당 시각 원소 추가
                            date_list.append(int(h)) # 해당 시각이 그날 48개로 분할된 시간 중 몇번째인지 원소 추가
                            temp = driver.find_element(By.XPATH,'//*[@id="inner-content"]/div[2]/div[1]/div[5]/div[1]/div/lib-city-history-observation/div/div[2]/table/tbody/tr['+str(h2)+']/td[2]')
                            temp_str = temp.text # 스크래핑한 xpath의 온도를 str로 변경
                            temp_str.split
                            temp_t=temp_str[:2] # temp_str 에서 온도 숫자만 추출
                            temp_c=temp_str[2:]
                            temp_t_x = temp_t
                            temp_c_x = temp_c
                            temp_list = date_list 
                            temp_list.append(temp_t) # 날짜+시각 리스트에 온도 원소 추가
                            temp_list.append(temp_c) # 날짜+시각 리스트에 섭씨 기호 원소 추가
                            time.sleep(0.02)
                            print(temp_list) # 확인용 print
                            # 시간 루프 종료 지점

                            # csv 추가 및 저장
                            with open("./Weather_Data_CSV/"+str(y)+'_'+str(m)+'_'+"London_Temperature.csv", 'a', newline='') as file:
                                writer = csv.writer(file)
                                writer.writerow(temp_list)

                            print()
                            h2 = h2+1
                        
                        else:
                            try:
                                date_list = [int(y), int(m), int(d)]
                                print('결손 데이터 칸의 정상 시각 >>',re_clk_11_48[h])
                                print('사이트에서 스크래핑한 시각 >>', clk_str)
                                date_list.append(re_clk_11_48[h])
                                date_list.append(int(h))
                                na_list = date_list
                                na_list.append(temp_t_x) # 직전 시각의 온도를 그대로 입력
                                na_list.append(temp_c_x)
                                print(na_list)

                                with open("./Weather_Data_CSV/"+str(y)+'_'+str(m)+'_'+"London_Temperature.csv", 'a', newline='') as file:
                                    writer = csv.writer(file)
                                    writer.writerow(na_list)
                                print() 
                            except KeyError:
                                break
                        h = h+1
                    
                    else:
                        for E in range(1):
                            h2 = h2+1

                except NoSuchElementException:
                    if(h<49):
                        # 맨 마지막에 데이터가 없어 조기 종료 되버리는 경우
                        date_list = [int(y), int(m), int(d)]
                        print('결손 데이터 칸의 정상 시각 >>',re_clk_11_48[h])
                        print('사이트에서 스크래핑한 시각 >>', clk_str)
                        date_list.append(re_clk_11_48[h])
                        date_list.append(int(h))
                        na_list = date_list
                        na_list.append(temp_t)
                        na_list.append(temp_c)
                        print(na_list)
                            
                        with open("./Weather_Data_CSV/"+str(y)+'_'+str(m)+'_'+"London_Temperature.csv", 'a', newline='') as file:
                            writer = csv.writer(file)
                            writer.writerow(na_list)
                            print() 
                        h = h+1
                        
                    else:
                        break
                        
        temp_t_x = temp_t                
        print('< Day End >')
        print('\n'*4)  

#### 2013~2016년의 기상 데이터를 수집하는 전체 코드

In [14]:
# 데이터 저장을 위한 csv 
import csv

# Selenium : 
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

# 에러 예외처리를 위해 해당 예외를 먼저 임포트해줌.
from selenium.common.exceptions import NoSuchElementException

# 정상작동을 위해서 중간 시간 정보 호출 사이에 강제 딜레이를 잠깐 넣어줌.
import time

path = "./chromedriver.exe"
driver = webdriver.Chrome(path)
driver.implicitly_wait(10)

# 기상포털 주소 1차 접속. 데이터가 충분히 표시된 2021년 1월 1일을 시작 화면으로 설정
driver.get('https://www.wunderground.com/history/daily/gb/london/EGLC/date/2021-1-1')
time.sleep(1)

# 온도 표기 선택창 클릭
driver.find_element(By.XPATH, '//*[@id="wuSettings"]/i').click()
time.sleep(1)

# 섭씨 선택
driver.find_element(By.XPATH, '//*[@id="wuSettings-quick"]/div/a[2]').click()
time.sleep(1)

# YYYY offset 2013
y = 2013
while(y<2017):
    print('====================< %d년 >===================================' %y)
    
    # MM offset 1
    m = 1
    while(m<13):
        
        print('====================< %d월 >==============================' %m)
        
        # DD
        Monthly_Temp(y, m)
        
        temp_t_x = temp_t
        temp_c_x = temp_c
        m=m+1              
        print()
    print('\n')
    
    """
    여기서 당해의 1년 csv 데이터 합본 생성 (2013, 2014, 2015, 2016)
    2013_1_London_Temperature 파일 이름에서
    2013_m_London_Temperature 로 해서 m이 1~12 범위 내에서 반복 결합 되도록
    파일이름은 2013_Unit_London_Temperature
    """
    y=y+1

print('스크래핑 종료.')
driver.quit()

C:\Users\Hwi\AppData\Local\Temp/ipykernel_8724/1542880815.py:16: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(path)


====================< 2013년 >===================================
====================< 1월 >==============================
====================< 1일 >=========================

[2013, 1, 1, '11:50 PM', 1, '10', ' °C']

[2013, 1, 1, '12:20 AM', 2, '10', ' °C']

[2013, 1, 1, '12:50 AM', 3, '9 ', '°C']

[2013, 1, 1, '1:20 AM', 4, '9 ', '°C']

[2013, 1, 1, '1:50 AM', 5, '9 ', '°C']

[2013, 1, 1, '2:20 AM', 6, '8 ', '°C']

[2013, 1, 1, '2:50 AM', 7, '8 ', '°C']

[2013, 1, 1, '3:20 AM', 8, '8 ', '°C']

[2013, 1, 1, '3:50 AM', 9, '8 ', '°C']

[2013, 1, 1, '4:20 AM', 10, '8 ', '°C']

[2013, 1, 1, '4:50 AM', 11, '7 ', '°C']

[2013, 1, 1, '5:20 AM', 12, '7 ', '°C']

[2013, 1, 1, '5:50 AM', 13, '6 ', '°C']

[2013, 1, 1, '6:20 AM', 14, '6 ', '°C']

[2013, 1, 1, '6:50 AM', 15, '6 ', '°C']

[2013, 1, 1, '7:20 AM', 16, '5 ', '°C']

[2013, 1, 1, '7:50 AM', 17, '5 ', '°C']

[2013, 1, 1, '8:20 AM', 18, '5 ', '°C']

[2013, 1, 1, '8:50 AM', 19, '5 ', '°C']

[2013, 1, 1, '9:20 AM', 20, '5 ', '°C']

[2013, 1,

KeyboardInterrupt: 

#### 1년 기준 12달치 통합 저장 코드

In [ ]:
# 보류